In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords

<WordListCorpusReader in '/root/nltk_data/corpora/stopwords'>

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.3 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3122008 sha256=2bf0ff0f54b20678388cd1f9befe23106dc5a9f26029c7f4bf099315e52bbe1b
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import fasttext
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import re

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv('test.csv')

In [ ]:
train.head()
'''
base_score is a continunous real value, so it is a regression problem
there are 7 features, a combination of text and numberic data
I need to find a way to convert all the data into numerical vector that can be used as input for training and testing
patient_id, effictiveness_rating, number _of_times_prescribed are already numbers so dont need embedding
I will use fasttext to embed the text of name_of_drug,	use_case_for_drug,	review_by_patient
drug_approved_by_UIC are date, can be converted to numerical format
For the final input vector X, I need to combine the text and numeriacl data into one vector.
For that I decided to do "direct sum" of the vectors, that considers each new feature as a new Dimension for the input vector
'''


,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [ ]:
# checking of there are any bad null values in the data
train.isnull().sum()

patient_id                    0
name_of_drug                  0
use_case_for_drug             0
review_by_patient             0
effectiveness_rating          0
drug_approved_by_UIC          0
number_of_times_prescribed    0
base_score                    0
dtype: int64

In [ ]:
test.isnull().sum()

patient_id                    0
name_of_drug                  0
review_by_patient             0
drug_approved_by_UIC          0
number_of_times_prescribed    0
use_case_for_drug             0
effectiveness_rating          0
dtype: int64

In [ ]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens


def tokenize_and_remove_punctuation(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    return tokens


def remove_stopwords(word_tokens):
    filtered_tokens = [word for word in word_tokens if word not in stopwords.words('english')]
    return filtered_tokens


'''
Convert to lower case,
remove punctuation
lemmatize
'''
def preprocess_main(sent):
    sent = sent.lower()
    re.sub(' +', ' ', sent)
    re.sub('\n', ' ', sent)
    tokens = tokenize_and_remove_punctuation(sent)
    lemmatized_tokens = lemmatize_sentence(tokens)
    orig = lemmatized_tokens
    filtered_tokens = remove_stopwords(tokens)
    if len(filtered_tokens) == 0:
        # if stop word removal removes everything, don't do it
        filtered_tokens = orig
    normalized_sent = " ".join(filtered_tokens)
    return normalized_sent

In [ ]:
#make X_train, X_test
drugname_train = train['name_of_drug']
usecase_train = train['use_case_for_drug']
review_train = train['review_by_patient']
norm_review_train = review_train.replace()
date_train = train.drug_approved_by_UIC



In [ ]:
#to normalise statements in review
def normalise_series(series):
  for review in series:
    norm_review = preprocess_main(review)
    series = series.replace([review], norm_review)
  return series

In [ ]:
review_train = normalise_series(review_train)

In [ ]:
review_train.head()

0    side effect take combination bystolic 5 mg fis...
1    son halfway fourth week intuniv became concern...
2    used take another oral contraceptive 21 pill c...
3    suboxone completely turned life around feel he...
4    2nd day 5mg started work rock hard erections h...
Name: review_by_patient, dtype: object

In [ ]:
# test is the input for the final submission
drugname_test = test['name_of_drug']
usecase_test = test['use_case_for_drug']
review_test = test['review_by_patient']

date_test = test['drug_approved_by_UIC']

In [ ]:
review_test = normalise_series(review_test)

In [ ]:
# I am making a custom fasttext word embedding model for our dataset, with all the text i name_of_drug, use_case_for_Drug and review_by_patient
train_text = pd.DataFrame([drugname_train, usecase_train, review_train]).transpose()

In [ ]:
train_text.head()

,name_of_drug,use_case_for_drug,review_by_patient
0,Valsartan,Left Ventricular Dysfunction,side effect take combination bystolic 5 mg fis...
1,Guanfacine,ADHD,son halfway fourth week intuniv became concern...
2,Lybrel,Birth Control,used take another oral contraceptive 21 pill c...
3,Buprenorphine / naloxone,Opiate Dependence,suboxone completely turned life around feel he...
4,Cialis,Benign Prostatic Hyperplasia,2nd day 5mg started work rock hard erections h...


In [ ]:
import datetime
def embed_dates(date_series):
  dates = []
  for strdate in date_series:
    datetimeobj=datetime.datetime.strptime(strdate,"%d-%b-%y")
    date_numeric = 10000*datetimeobj.year + 100*datetimeobj.month + datetimeobj.day
    dates.append(date_numeric)
  return dates

In [ ]:
# this has name_of_drug, use_case_for_Drug and review_by_patient which will be fed to fasttext unsupervised training
train_text.to_csv('text_train.csv', index=False)

In [ ]:
model = fasttext.train_unsupervised('text_train.csv', model='skipgram', lr=0.05, dim=100, ws=5, epoch=5)
# now this model is trained on our medical dataset, and will be able to predict our field-specific data bettter than basic english pretrained fasttext model

In [ ]:
def get_fastext_vectors(series):
  vectors = []
  for item in series:
        vectors.append(model.get_sentence_vector(item))
  return vectors

In [ ]:
drugname_vec_train = get_fastext_vectors(drugname_train)
usecase_vec_train = get_fastext_vectors(usecase_train)
review_vec_train = get_fastext_vectors(review_train)
date_vec_train = embed_dates(date_train)

In [ ]:
drugname_vec_test = get_fastext_vectors(drugname_test)
usecase_vec_test = get_fastext_vectors(usecase_test)
review_vec_test = get_fastext_vectors(review_test)
date_vec_test = embed_dates(date_test)

In [ ]:
drugname_vec_train = np.array(drugname_vec_train)
usecase_vec_train = np.array(usecase_vec_train)
review_vec_train = np.array(review_vec_train)
date_vec_train = np.array(date_vec_train)

In [ ]:
drugname_vec_test = np.array(drugname_vec_test)
usecase_vec_test = np.array(usecase_vec_test)
review_vec_test = np.array(review_vec_test)
date_vec_test = np.array(date_vec_test)

In [ ]:
# here we can see the size pf the fasttext embedded vectors of each text docs are default 100
print(np.shape(drugname_vec_train))
print(np.shape(usecase_vec_train))
print(np.shape(review_vec_train))
print(np.shape(date_vec_train))
len(drugname_vec_train)

(32165, 100)
(32165, 100)
(32165, 100)
(32165,)


32165

In [ ]:
print(np.shape(drugname_vec_test))
print(np.shape(usecase_vec_test))
print(np.shape(review_vec_test))
print(np.shape(date_vec_test))
len(drugname_vec_test)

(10760, 100)
(10760, 100)
(10760, 100)
(10760,)


10760

In [ ]:
np.array(train['patient_id'][0])

array([206461,  95260,  92703, ..., 121154,  45410, 187382])

In [ ]:
# We are doing direct sum of all the numerical vectors, treating each feature as a seperate dimension
X_train = []
for i in range (0, len(drugname_vec_train)):
  Xi = np.concatenate(( np.array(train['patient_id'][i]), drugname_vec_train[i], usecase_vec_train[i], review_vec_train[i], np.array(train['effectiveness_rating'][i]), date_vec_train[i], np.array(train['number_of_times_prescribed'][i])), axis = None)
  X_train.append(Xi)

In [ ]:
X_test = []
for i in range (0, len(drugname_vec_test)):
  Xi = np.concatenate(( np.array(test['patient_id'][i]), drugname_vec_test[i], usecase_vec_test[i], review_vec_test[i], np.array(test['effectiveness_rating'][i]), date_vec_test[i], np.array(test['number_of_times_prescribed'][i])), axis = None)
  X_test.append(Xi)

In [ ]:
# here we can see the final size of each vector is 304, which makes sense
# because there are three text columns, each of size 100 (fasttext embedding), date, patient id, effectivess, number of times prescribed are 4 single numeric values 
# hence our input vectors with combined text and numerical data is ready 
np.shape(X_train)

(32165, 304)

In [ ]:
np.shape(X_test)

(10760, 304)

In [ ]:
# this is the desired output we wat to predict
y_train = np.array(train.base_score)

In [ ]:
y_test = np.array(test.)

In [ ]:
np.shape(y_train)

(32165,)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# I have used RFC though a number of other algorithms can be used and compared to find the best performer
regressor=RandomForestRegressor(max_features='sqrt')
regressor.fit(X_train,y_train)

RandomForestRegressor(max_features='sqrt')

In [ ]:
pred = regressor.predict(X_test)

In [ ]:
len(pred)

10760

In [ ]:
test.shape

(10760, 7)

In [ ]:
# checking random prediction
pred[1008]

6.383335234900478

In [ ]:
# sample output is rounded to 2 decimal places
pred = [prediction.round(2) for prediction in pred]

In [ ]:
# prepare final submission
submission = pd.DataFrame()
submission['patient_id'] = test['patient_id']
submission['base_score'] = pred
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

,patient_id,base_score
0,163740,7.39
1,39293,7.07
2,208087,6.38
3,23295,6.16
4,97013,6.38


In [ ]:
# download final submission
submission.to_csv('submission_file.csv', index=False)